In [ ]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import datetime
from sunpy.io import jp2
import numpy as np
import os
import sys
from PIL import Image

In [ ]:
pathofAIAFile = '/data/HMI_Fits/files-all.txt'
#Importing the file
df = pd.read_csv(pathofAIAFile, sep=" ", header=None)
df.columns = ['path']
df

In [ ]:
#Removing the links which don't have .fits extension
df['isFile'] = df['path'].progress_apply(lambda x: True if '.fits' in x else False)
img = df[df.isFile == True]

In [ ]:
#Adding /data/AIA to complete the path
img['path'] = img['path'].progress_apply(lambda x: x.replace('./', '/data/HMI_Fits/'))
img

In [ ]:
#b = img['year'].unique()
img['year']= img['path'].progress_apply(lambda x: x.split("HMI_Fits/")[1][0:4])
img['notYear'] = img['year'].progress_apply(lambda x: True if x in 'BadFits' else False)
img['notYear'] = img['year'].progress_apply(lambda x: True if x in "synt" else False)
searchfor = ['BadF', 'synt']
newdf = img[~img.year.str.contains('|'.join(searchfor))]
newdf['year'].unique()
final = newdf[['path']]
final = final.reset_index(drop=True)
#final.to_csv('FITS_image_data.csv', sep='\t', index=False)
final

In [ ]:
from astropy.io import fits
import torch
import numpy as np
from collections import Counter
import pandas
from astropy.io import fits
import matplotlib.pyplot as plt
filepath = final.iloc[0]['path']
img = fits.open(filepath)
data_a = img[0].data
#where_are_NaNs = np.isnan(data_a)
#data_a[where_are_NaNs] = 0
data_a = np.clip(data_a,-1500,1500)
print(data_a.shape[1])
plt.imshow(data_a, cmap='gray')
plt.colorbar()
def load_img(filepath):
    image_data = fits.getdata(filepath)
    data = image_data.astype(np.float32)
    a = torch.from_numpy(data)
    where_are_NaNs = np.isnan(a)
    a[where_are_NaNs] = 0
    a = torch.clamp(a,-1500,1500)
    return a
a = load_img(filepath)
data_a.max()

In [ ]:
counts,values = pd.Series(lis).value_counts().values, pd.Series(lis).value_counts().index
df_results = pd.DataFrame(list(zip(values,counts)),columns=["value","count"])
df_results.set_index('value').plot(figsize=(10,5), grid=True)

In [ ]:
print(len(lis))
data = pd.DataFrame(lis)
print(data)
stat = data.describe()
stat
jj = set(lis)
jj

In [ ]:
df_results = df_results.drop(df_results.index[0])
print(df_results)
#df_results.set_index('value').plot(figsize=(10,5), grid=True)

In [ ]:
#jj = set(lis)
fresh = []
for i in range(1,76241):
    filepath = final.iloc[i]['path']
    a = load_img(filepath)
    fresh = [element.item() for element in a.flatten()]
    lis = lis+fresh